# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [7]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [8]:
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

Epoch 137/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [===========

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [9]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [10]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.5486 - mse: 0.5486 - val_loss: 0.1954 - val_mse: 0.1954
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2501 - mse: 0.2501 - val_loss: 0.1487 - val_mse: 0.1487
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2078 - mse: 0.2078 - val_loss: 0.1345 - val_mse: 0.1345
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1718 - mse: 0.1718 - val_loss: 0.1249 - val_mse: 0.1249
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1556 - mse: 0.1556 - val_loss: 0.1272 - val_mse: 0.1272
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1376 - mse: 0.1376 - val_loss: 0.1145 - val_mse: 0.1145
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1295 - mse: 0.1295 - val_loss: 0.1141 - val_mse: 0.1141
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1160 - m

Epoch 63/150
33/33 [==============================] - ETA: 0s - loss: 0.0215 - mse: 0.021 - 0s 1ms/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.1009 - val_mse: 0.1009
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1015 - val_mse: 0.1015
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0235 - mse: 0.0235 - val_loss: 0.1006 - val_mse: 0.1006
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1018 - val_mse: 0.1018
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1005 - val_mse: 0.1005
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1019 - val_mse: 0.1019
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0218 - mse: 0.0218 - val_loss: 0.1006 - val_mse: 0.1006
Epoch 70/150
33/33 [==================

33/33 [==============================] - 0s 1ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.0976 - val_mse: 0.0976
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.0971 - val_mse: 0.0971
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.0969 - val_mse: 0.0969
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.0973 - val_mse: 0.0973
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.0981 - val_mse: 0.0981
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.0990 - val_mse: 0.0990
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.0980 - val_mse: 0.0980
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0108 -

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [11]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 755us/step - loss: 0.0087 - mse: 0.0087


[0.008664648979902267, 0.008664648979902267]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [12]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 958us/step - loss: 0.0964 - mse: 0.0964


[0.09636756032705307, 0.09636756032705307]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [13]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

24394.78194463418

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [14]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.4392 - mse: 0.4392 - val_loss: 0.1985 - val_mse: 0.1985
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2410 - mse: 0.2410 - val_loss: 0.1697 - val_mse: 0.1697
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2015 - mse: 0.2015 - val_loss: 0.1610 - val_mse: 0.1610
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1820 - mse: 0.1820 - val_loss: 0.1377 - val_mse: 0.1377
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1528 - mse: 0.1528 - val_loss: 0.1382 - val_mse: 0.1382
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1432 - mse: 0.1432 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1285 - mse: 0.1285 - val_loss: 0.1307 - val_mse: 0.1307
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1233 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0274 - mse: 0.0274 - val_loss: 0.1156 - val_mse: 0.1156
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0272 - mse: 0.0272 - val_loss: 0.1059 - val_mse: 0.1059
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0268 - mse: 0.0268 - val_loss: 0.1199 - val_mse: 0.1199
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0263 - mse: 0.0263 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0257 - mse: 0.0257 - val_loss: 0.1064 - val_mse: 0.1064
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.1062 - val_mse: 0.1062
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0256 - mse: 0.0256 - val_loss: 0.1082 - val_mse: 0.1082
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0255 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0141 - mse: 0.0141 - val_loss: 0.1092 - val_mse: 0.1092
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.1083 - val_mse: 0.1083
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.1086 - val_mse: 0.1086
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0136 - mse: 0.0136 - val_loss: 0.1108 - val_mse: 0.1108
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0132 - mse: 0.0132 - val_loss: 0.1096 - val_mse: 0.1096
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0130 - mse: 0.0130 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0130 - mse: 0.0130 - val_loss: 0.1101 - val_mse: 0.1101
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 748us/step - loss: 0.0104 - mse: 0.0104


[0.010434646159410477, 0.010434646159410477]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 934us/step - loss: 0.1114 - mse: 0.1114


[0.11143090575933456, 0.11143090575933456]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [17]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 6ms/step - loss: 0.4691 - mse: 0.4691 - val_loss: 0.1860 - val_mse: 0.1860
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2328 - mse: 0.2328 - val_loss: 0.1520 - val_mse: 0.1520
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1858 - mse: 0.1858 - val_loss: 0.1389 - val_mse: 0.1389
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1601 - mse: 0.1601 - val_loss: 0.1393 - val_mse: 0.1393
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1375 - mse: 0.1375 - val_loss: 0.1335 - val_mse: 0.1335
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1184 - mse: 0.1184 - val_loss: 0.1283 - val_mse: 0.1283
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1089 - mse: 0.1089 - val_loss: 0.1214 - val_mse: 0.1214
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0997 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1384 - val_mse: 0.1384
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.1377 - val_mse: 0.1377
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.1398 - val_mse: 0.1398
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0202 - mse: 0.0202 - val_loss: 0.1370 - val_mse: 0.1370
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0203 - mse: 0.0203 - val_loss: 0.1371 - val_mse: 0.1371
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0195 - mse: 0.0195 - val_loss: 0.1397 - val_mse: 0.1397
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0195 - mse: 0.0195 - val_loss: 0.1431 - val_mse: 0.1431
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0193 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1454 - val_mse: 0.1454
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1457 - val_mse: 0.1457
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1483 - val_mse: 0.1483
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1446 - val_mse: 0.1446
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.1448 - val_mse: 0.1448
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.1457 - val_mse: 0.1457
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.1460 - val_mse: 0.1460
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 636us/step - loss: 0.0065 - mse: 0.0065


[0.006512728054076433, 0.006512728054076433]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 776us/step - loss: 0.1485 - mse: 0.1485


[0.1484544575214386, 0.1484544575214386]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [20]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.3621 - mse: 0.3621 - val_loss: 0.1583 - val_mse: 0.1583
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1882 - mse: 0.1882 - val_loss: 0.1510 - val_mse: 0.1510
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1332 - mse: 0.1332 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1091 - mse: 0.1091 - val_loss: 0.1464 - val_mse: 0.1464
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0778 - mse: 0.0778 - val_loss: 0.1178 - val_mse: 0.1178
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0661 - mse: 0.0661 - val_loss: 0.2033 - val_mse: 0.2033
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0573 - mse: 0.0573 - val_loss: 0.1182 - val_mse: 0.1182
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0416 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0164 - mse: 0.0164 - val_loss: 0.1182 - val_mse: 0.1182
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0075 - mse: 0.0075 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1153 - val_mse: 0.1153
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.1036 - val_mse: 0.1036
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1369 - val_mse: 0.1369
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0060 - mse: 0.0060 - val_loss: 0.1162 - val_mse: 0.1162
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0074 - mse: 0.0074 - val_loss: 0.0952 - val_mse: 0.0952
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0126 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.1084 - val_mse: 0.1084
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071 - val_loss: 0.1053 - val_mse: 0.1053
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.1000 - val_mse: 0.1000
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.1059 - val_mse: 0.1059
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0031 - mse: 0.0031 - val_loss: 0.1058 - val_mse: 0.1058
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.1133 - val_mse: 0.1133
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.1033 - val_mse: 0.1033
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 765us/step - loss: 0.0026 - mse: 0.0026


[0.0025758366100490093, 0.0025758366100490093]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 779us/step - loss: 0.1008 - mse: 0.1008


[0.10076664388179779, 0.10076664388179779]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [23]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 11ms/step - loss: 0.4221 - mse: 0.4221 - val_loss: 0.1451 - val_mse: 0.1451
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1709 - mse: 0.1709 - val_loss: 0.1167 - val_mse: 0.1167
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1071 - mse: 0.1071 - val_loss: 0.1016 - val_mse: 0.1016
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0766 - mse: 0.0766 - val_loss: 0.1006 - val_mse: 0.1006
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0624 - mse: 0.0624 - val_loss: 0.1114 - val_mse: 0.1114
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0488 - mse: 0.0488 - val_loss: 0.1095 - val_mse: 0.1095
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0399 - mse: 0.0399 - val_loss: 0.1146 - val_mse: 0.1146
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0311 - 

33/33 [==============================] - 0s 2ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1131 - val_mse: 0.1131
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1122 - val_mse: 0.1122
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 7.7791e-04 - mse: 7.7791e-04 - val_loss: 0.1171 - val_mse: 0.1171
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 6.5807e-04 - mse: 6.5807e-04 - val_loss: 0.1116 - val_mse: 0.1116
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 4.6189e-04 - mse: 4.6189e-04 - val_loss: 0.1165 - val_mse: 0.1165
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 4.0511e-04 - mse: 4.0511e-04 - val_loss: 0.1125 - val_mse: 0.1125
Epoch 70/150
33/33 [==============================] - 0s 

Epoch 124/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1032 - val_mse: 0.1032
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1084 - val_mse: 0.1084
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 8.8196e-04 - mse: 8.8196e-04 - val_loss: 0.1042 - val_mse: 0.1042
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 9.9765e-04 - mse: 9.9765e-04 - val_loss: 0.1040 - val_mse: 0.1040
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1113 - val_mse: 0.1113
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1081 - val_mse: 0.1081
Epoch 131/150
33/33 [==============================] 

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 785us/step - loss: 0.0087 - mse: 0.0087


[0.008665170520544052, 0.008665170520544052]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 886us/step - loss: 0.1182 - mse: 0.1182


[0.1181960329413414, 0.1181960329413414]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [26]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 997us/step - loss: 0.1281 - mse: 0.1281


[0.12808018922805786, 0.12808018922805786]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [27]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

28123.699401480855

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.